In [14]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import os
import json

In [18]:
# open config file
def load_config(config_path=None):
    if config_path is None:
        script_dir = os.path.dirname(__file__) 
        project_root = os.path.abspath(os.path.join(script_dir, '..', '..'))  # navigate two levels up to the project root
        config_path = os.path.join(project_root, 'config.json')
    
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    return config

def load_data(data_path):
    data = pd.read_csv(data_path)
    return data

In [24]:
import os
import json
import pandas as pd

# open config file
def load_config(config_path=None):
    if config_path is None:
        # Get the current working directory
        cwd = os.getcwd()
        project_root = os.path.abspath(os.path.join(cwd, '..'))  # navigate two levels up to the project root
        config_path = os.path.join(project_root, 'config.json')
    
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    return config

def load_data(data_path):
    data = pd.read_csv(data_path)
    return data

config = load_config()
data_path = config['data_path']
data = load_data(data_path)

In [29]:
print(data.head())

    slowdelta   fastdelta   slowtheta   fasttheta      alpha       beta  \
0  528.584023  370.200447  235.735391  205.805979  75.988092  13.273196   
1  368.691405  214.083349  165.467827   92.983005  41.392088   7.643469   
2  148.764327  405.340661  235.939887   94.879132  42.856426  10.731685   
3   64.648625   85.934535  200.231794  163.555837  29.363525   5.409392   
4   69.056042   91.792258  303.882162   56.034816  19.103509   5.254522   

      gamma       rms  mouseID  lab  sleepstage  epoch  
0  1.274694  0.464923      1.0  1.0         3.0    1.0  
1  0.786264  0.342968      1.0  1.0         3.0    2.0  
2  1.135943  0.315582      1.0  1.0         3.0    3.0  
3  1.036426  0.294122      1.0  1.0         1.0    6.0  
4  1.065458  0.288047      1.0  1.0         1.0    7.0  
